# DashStyle Fusion
#### Apply the style of one ArcGIS Dashboard to other ArcGIS Dashboards
created by Niklas Köhn, Esri Deutschland, 2024

In [ ]:
from arcgis.gis import GIS
import getpass

# user input username + password
username = input("Username: ")
password = getpass.getpass(prompt='Password: ', stream=None) # hide password

# login to ArcGIS Online or ArcGIS Enterprise
gis = GIS("https://www.arcgis.com", username, password)
print("Successfully logged in as: " + gis.properties.user.username)

### Search for dashboards

In [ ]:
# user select to search org or own account
print("1. Search in the organization")
print("2. Search in my account")
choice = input("Enter your choice: ")

# search for dashboards in the org or own account
outside_org_choice = True
if choice == "1":
    outside_org_choice = False

search = input("Enter search term (empty string to search for all dashboards): ")
items = gis.content.search(query=search, item_type="Dashboard", max_items=100, outside_org=outside_org_choice)
print("Found " + str(len(items)) + " dashboards")

for item in items:
    print(item.title + " (" + item.id + ")")

### Copy all dashboards to folder in own account

In [ ]:
import datetime

# create folder in own account named "dashboards_<date>_time"
folder_name = "dashboards_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
folder = gis.content.create_folder(folder_name)

# copy all dashboards to the folder
copied_items = []
for item in items:
    # use arcgis python api to copy dashboard item. folder parameter does not work, everything ends up in root :(
    copied_item = item.copy_item(folder=folder_name)
    copied_items.append(copied_item)
    print ("Dashboard " + copied_item + " copied.")

# workaround because folder parameter doesn't work: move items. if this fails, delete copies.
moved_items = []
deleted_items = []
for item in copied_items:
    objMoved = item.move(folder=folder_name)  #folder not found for given owner message if it doesnt move it.
    moved_items.append(item)
    if (objMoved is None) or \
            (objMoved is not None and 'success' in objMoved and not objMoved['success']):
        deleted_items.append(item)
        item.delete(force=False, dry_run=False)
        sys.exit(
            "ERROR: Unable to move the item with ItemID {0} to the folder, {1}. ".format(
                id, folder_name))
    
print ("Copied " + str(len(moved_items)) + " dashboards to folder: " + folder_name)
for item in moved_items:
    print(item.title + " (" + item.id + ")")

### Select template

In [ ]:
# select the template dashboard from the copied items
template = folder.search(query="title:template", item_type="Dashboard")[0]

# show preview of the template dashboard
template.preview()

# # show template dashboard
# template.open_dashboard()

### Apply template header and theme

In [ ]:
# get template dashboard JSON
template_json = template.get_data()

# copy header section from template_json
header = template_json["header"]

# copy theme property from template_json
theme = template_json["theme"]

# copy themeOverrides from template_json
themeOverrides = template_json["themeOverrides"]

print (template_json, "\n\n ... \n\n", header, "\n\n ... \n\n", theme, "\n\n ... \n\n", themeOverrides)